In [3]:
%matplotlib inline

import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'src/')))

from sgpi import model

import sqlalchemy as sa
from sqlalchemy import orm, func, or_, and_

from sgpi.plot import *

import pandas as pd
import pysgpp as sg

In [4]:
# First get the estimators with the best CV-MSE from the database
ses = model.make_session()
def get_results(ses, level, dataset):
    return ses.query(model.Result, func.min(model.Result.validation_mse)).\
        filter(model.Result.experiment_id.between(51,100)).\
        join(model.RegularizationConfig).\
        join(model.GridConfig).\
        join(model.Experiment).\
        join(model.AdaptivityConfig).\
        filter(model.GridConfig.level == level).\
        filter(or_(and_(model.RegularizationConfig.exponent_base == 1.0, model.RegularizationConfig.type == sg.RegularizationType_Diagonal),\
                   model.RegularizationConfig.type == sg.RegularizationType_Identity)).\
        filter(model.RegularizationConfig.exponent_base == 1.0).\
        filter(model.AdaptivityConfig.no_points == 3).\
        filter(model.AdaptivityConfig.num_refinements == 5).\
        filter(model.Experiment.dataset == dataset).\
        group_by(model.GridConfig.T)


In [5]:
def print_latex(ses, level, dataset):
    for res, mse in get_results(ses, level=level, dataset=dataset):
        print "{} & {:1.4e} & {}({:4.0f}) & {:2.3f} & {} & {:2.3f} & {:2.3f}\\\\".\
            format(res.grid_config.T, res.regularization_config.lambda_reg,
                   res.validation_grid_points_mean,res.validation_grid_points_stdev*10,
                   np.sqrt(res.validation_mse), res.train_grid_points, np.sqrt(res.train_mse), np.sqrt(res.test_mse))

In [6]:
print_latex(ses, level=4, dataset='friedman1')

-0.5 & 2.2762e-10 & 5541.3( 185) & 1.246 & 5547.0 & 0.823 & 1.226\\
0 & 1.4539e-04 & 2278.8(  33) & 1.196 & 2277.0 & 0.845 & 1.179\\
0.5 & 7.7081e-05 & 640.8( 141) & 1.051 & 651.0 & 0.959 & 1.028\\
1 & 1.0432e-04 & 391.2(  72) & 1.031 & 395.0 & 0.976 & 1.015\\


In [7]:
print_latex(ses, level=5, dataset='concrete')

-0.4 & 1.9276e-02 & 8468.7( 206) & 4.703 & 8470.0 & 2.275 & 4.215\\
0 & 1.9622e-02 & 6678.3( 271) & 4.709 & 6650.0 & 2.286 & 4.184\\
0.5 & 6.2935e-03 & 1140.4( 239) & 4.771 & 1180.0 & 2.664 & 3.797\\
0.6 & 1.2700e-02 & 712.7( 242) & 4.781 & 685.0 & 3.398 & 4.308\\
1 & 1.0149e-02 & 517.9( 233) & 4.929 & 516.0 & 3.628 & 4.508\\


In [35]:
best_for_t = ses.query(model.Result, func.min(model.Result.validation_mse)).\
    join(model.Experiment).\
    join(model.GridConfig).\
    join(model.RegularizationConfig).\
    group_by(model.GridConfig.T).\
    group_by(model.GridConfig.level).\
    filter(model.Experiment.dataset=='concrete').\
    filter(model.GridConfig.level == 4).\
    filter(model.GridConfig.T == 0).\
    filter(or_(and_(model.RegularizationConfig.exponent_base == 1.0, model.RegularizationConfig.type == sg.RegularizationType_Diagonal),\
                   model.RegularizationConfig.type == sg.RegularizationType_Identity)).\
    order_by(model.GridConfig.level, model.Result.validation_mse)

In [36]:
best_for_t.count()

1

In [44]:
for res, mse in best_for_t:
    print res.grid_config.T, res.grid_config.level, np.sqrt(res.validation_mse), res.train_grid_points, res.adaptivity_config.num_refinements,\
        res.adaptivity_config.no_points

0 4 5.00738579498 1470.0 5 3
